In [6]:
#Updates vector S in order according to the
#synchronous update rule
using MNIST

function synchupdate!(W::Array, S::Array)
    b = copy(S)
    #S = 2*S - 1
    S = S.>0
    return S
end

function convertbool(val::Bool)
    return (val==true) ? 1 : -1
end

function convertbool2(val::Bool)
    return (val==true) ? 1 : 0
end

#Takes an image vector containing float values
# and returns an image vector of only true and false values
function setupimage(image)
    a = Bool[]
    for j = 1:784
        val = image[j] / 256
        push!(a, (val > 0.5) ? true : false)
    end
    push!(a, true)
    return a
end

function store(Xi::Array)
    #enforce dimensions of S to be 2
    if ndims(Xi) != 2
        println("Wrong number of dimensions in Xi")
        return
    end
    nitems = size(Xi, 1)# number of rows, each row one item
    N = size(Xi, 2) #number of columns
    W = zeros(N, N)
    for memoryitem = 1:nitems
        for i = 1:N
            for j = 1:N
                W[i, j] += convertbool(Xi[memoryitem,i]) * convertbool(Xi[memoryitem,j])
            end
        end
    end
    for i = 1:N
        W[i, i] = 0
    end
    return W
end

function findattractor(W, sigma)
    finish = false
    count = 0
    while !finish
        if count > 20 * length(sigma)
            print("failed to converge")
            return sigma
        end
        count += 1
        b = copy(sigma)
        sigma = synchupdate!(W, sigma)
        if sigma == b
            finish = true
        end
    end
    
    return sigma
end

function addtomemory(Mem::Matrix, Sigma::Vector)
    Mem = [Mem; Sigma] #Adds Sigma to Mem as its last row
    return Mem
end

function constructmem(n::Int)    #Create a memory matrix from the first n images in the training set   
    Mem = Matrix(0, 785)
    for i = 1:n
        image = setupimage(trainfeatures(i))
        image = reshape(image, 1, 785)
        Mem = [Mem; image]
    end
    return Mem     
end

function mod_mem()
    Mem = Matrix(0, 785)
    used = []
    i = 1
    while length(used) < 10 
        if !(trainlabel(i) in used)
            Mem = [Mem; reshape(setupimage(trainfeatures(i)), 1, 785)]
            push!(used, trainlabel(i))
        end
        i += 1
    end
    return imageset
end

#pixel_digits = mod_mem()



mod_mem (generic function with 1 method)

In [7]:
using MAT
using MNIST

hiddenunits = matopen("300codes.mat")
hiddenimages = read(hiddenunits, "BA_Z")

testvec = hiddenimages[4, :]
using Distances
W = store(hiddenimages)
testvec_copy = copy(testvec)
testvec = findattractor(W, testvec)
hamming(testvec, testvec_copy)


attractor = findattractor(W, hiddenimages[90, :])

function printattractor(att)
    str = "["
    for i = 1:length(att)
        if att[i] == false
            str = string(str, "0 ")
        else 
            str = string(str, "1 ")
        end
    end
    str = string(str, "]")
end

printattractor(findattractor(W, hiddenimages[90, :]))

"[1 0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 ]"

In [9]:
#Comparing against the first 120 images in the pixel representation
using Gadfly
mem = constructmem(120)
W = store(mem)
using Distances
test = mem[5, :]
test_copy = copy(test)
test = findattractor(W, test)
current = hamming(test, test_copy)



92

In [10]:
using MNIST
trainlabel(5)


9.0

In [53]:

using MAT
using MNIST

hiddenunits = matopen("J_with_8.mat")
hiddenimages = read(hiddenunits, "Jnew")
hiddenunits2 = matopen("500codes.mat")
hiddenimages2 = read(hiddenunits2, "BA_Z1")
hiddenunits3 = matopen("700codes.mat")
hiddenimages3 = read(hiddenunits3, "BA_Z3")
#imageset = hiddenimages[1:11, :]

function generate_set(images)
    used = []
    imageset = images[1, :]
    push!(used, trainlabel(1))
    i = 2
    while length(used) < 10 
        if !(trainlabel(i) in used)
            print(trainlabel(i))
            imageset = [imageset images[i, :]]
            push!(used, trainlabel(i))
        end
        i += 1
    end
    return imageset.'
end

function calcprecision(image_set::Array)
    tot = 0
    lowest = 300
    highest = 0
    for i in 1:size(image_set, 1)
        test = image_set[i, :]
        test_copy = copy(test)
        test = findattractor(W, test)
        current = hamming(test, test_copy)
        lowest = min(lowest, current)
        highest = max(highest, current)
        tot += current
    end
    avg = tot / size(image_set, 1)
    
    return (avg, lowest, highest)
end

imageset = generate_set(hiddenimages2)

W = store(imageset)

for i = 1:size(imageset, 1)
    test_vec = imageset[i, :]
    temp = copy(test_vec)
    test_vec = findattractor(W, test_vec)
    print(hamming(test_vec, temp))
    print(" ")
end
print("   |    ")

#W = store(pixel_digits)

#for i = 1:size(pixel_digits, 1)
#    test_vec = pixel_digits[i, :]
 #   temp = copy(test_vec)
  #  test_vec = findattractor(W, test_vec)
   # print(hamming(test_vec, temp))
    #print(" ")
#end


#printattractor(findattractor(W, new_test))
#trainlabel(201)
function closest(n, imageset)
    temp = hiddenimages2[n, :]
    W = store(imageset)
    test = findattractor(W, temp)
    for i = 1:10
        print(hamming(test, hiddenimages2[i, :]))
        print("  ")
    end
end
closest(97, imageset)

0.04.01.09.02.03.06.07.08.00 0 0 0 0 0 0 0 0 0    |    207  178  153  126  60  156  130  204  128  130  

In [55]:
function capacity(images, n)
    set = images[1:n, :]
    W = store(set)
    for i = 1:size(set, 1)
        test_vec = set[i, :]
        temp = copy(test_vec)
        test_vec = findattractor(W, test_vec)
        print(hamming(test_vec, temp))
        print("  ")
    end
end

function capacityavg(images, n)
    set = images[1:n, :]
    W = store(set)
    tot = 0
    for i = 1:size(set, 1)
        test_vec = set[i, :]
        temp = copy(test_vec)
        test_vec = findattractor(W, test_vec)
        tot += hamming(test_vec, temp)
    end
    return tot / n
end

capacity(hiddenimages, 7)
print("  |  ")
capacity(constructmem(10), 2)

LoadError: MethodError: no method matching convertbool(::Float64)[0m
Closest candidates are:
  convertbool([1m[31m::Bool[0m) at In[51]:14[0m

In [182]:
Pkg.add("Gadfly")
using Gadfly
plot(x = 1:3, y = [1 2 3])

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Gadfly
INFO: Use `Pkg.update()` to get the latest versions of your packages


LoadError: UndefVarError: plot not defined

In [54]:
function gencapacitydata(images, n, scale)
    data = []
    for i = 1:n
        pt = capacityavg(images, i) / scale
        push!(data, pt)
    end
    return data
end
mnistmems = constructmem(50)
print(gencapacitydata(hiddenimages, 50, 100))

LoadError: MethodError: no method matching convertbool(::Float64)[0m
Closest candidates are:
  convertbool([1m[31m::Bool[0m) at In[51]:14[0m

In [113]:
print(mean(hiddenimages3[12, :]), " ", var(hiddenimages3))

0.5 0.24999998054641848

In [110]:
cov(hiddenimages2[12, :], hiddenimages2[12, :])

0.25082497212932003

In [60]:
mnistmems[1, :]

785-element Array{Any,1}:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
  true

In [93]:
function generate_set_with_labels(images)
    used = []
    imageset = images[1, :]
    push!(used, trainlabel(1))
    i = 2
    while length(used) < 10 
        if !(trainlabel(i) in used)
            print(trainlabel(i))
            imageset = [imageset images[i, :]]
            push!(used, trainlabel(i))
        end
        i += 1
    end
    return [imageset.', used]
end

function classify(image, image_subset, labels, weights)
    
    
    attractor = findattractor(weights, image)
    min_differences = 1000
    best = 0
    for i in 1:10
        if hamming(image_subset[i, :], attractor) < min_differences
            best = labels[i]
            min_differences = hamming(image_subset[i, :], attractor)
        end
    end
    return best
end

function classify_multiple(images, start, last)
    pairs = []
    generated = generate_set_with_labels(images)
    image_subset = generated[1]
    labels = generated[2]
    weights = store(images)
    pair = 0
    for i in start:last
        guess = classify(images[i, :], image_subset, labels, weights)
        pair = (guess, trainlabel(i))
        push!(pairs, pair)
    end
    return pairs
end
    

#classify(hiddenimages3[300, :], 1)
classify_multiple(hiddenimages3, 50, 80)

0.04.01.09.02.03.06.07.08.0

31-element Array{Any,1}:
 (0.0,3.0)
 (1.0,3.0)
 (0.0,0.0)
 (0.0,7.0)
 (1.0,4.0)
 (0.0,9.0)
 (1.0,8.0)
 (0.0,0.0)
 (1.0,9.0)
 (0.0,4.0)
 (1.0,1.0)
 (0.0,4.0)
 (1.0,4.0)
 ⋮        
 (1.0,0.0)
 (0.0,0.0)
 (1.0,1.0)
 (1.0,7.0)
 (1.0,1.0)
 (1.0,6.0)
 (0.0,3.0)
 (0.0,0.0)
 (1.0,2.0)
 (1.0,1.0)
 (1.0,1.0)
 (1.0,7.0)

In [109]:
function num_digits(n)
    seen = 0
    for i = 1:2000
        if trainlabel(i) == n
            seen += 1
        end
    end
    return seen
end

num_digits(5)
    
function digits_in_set()
    for i = 0:9
        print(i)
        print(": ")
        print(num_digits(i))
        print("\n")
    end
end
digits_in_set()

0: 

191
1: 220
2: 198
3: 191
4: 214
5: 180
6: 200
7: 224
8: 172
9: 210
